In [1]:
#import necessary libraries
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd

In [2]:
#create a function that goes into the linkedin home page and signs in with email and password
def linkedInSignIn(your_email,your_password,job_url_of_interest):
    
    #navigating to website section
    try:
        #Set up the webdriver
        global driver
        
        driver = webdriver.Chrome()       

        # Navigate to the webpage
        driver.get("https://www.linkedin.com/home")

        #click into the email/phone number
        email = your_email

        #password of interest
        password = your_password
        
    except:
        "Could not get to the webpage"
        
    #email section
    try:
        #Find the button element using XPath
        email_button = driver.find_element(By.XPATH, '//*[@id="session_key"]')

        #put loggin information into box
        email_button.send_keys(email)

    except:
        print('xpath changed for email section')

    #password section
    try:
        #password of interest
        password_button = driver.find_element(By.XPATH, '//*[@id="session_password"]')
        
        #send password into input box
        password_button.send_keys(password)

    except:
        print('xpath changed for password section')


    #find the "Sign in" button element by XPath; 
    try:
        sign_in_button = driver.find_element(By.XPATH, '//div[@data-id="sign-in-form__footer"]//button')

        #click on the "Sign in" button
        sign_in_button.click()

    except:
        #print out an error message
        print('Xpath changed for the sign in button')
        
    
    try:
        #data science, united states,remote, and easy apply only
        driver.get(job_url_of_interest)

    except:
        print('going to linkedIn job page broke')

In [3]:
"""
this function should be updated so that it gets several lists of job elements.
include a for loop that controls the amount to scroll by
assumming the starting point is at 90% zoomed in, then we can scroll by 1,250 pixels until 

print(f"arguments[0].scrollBy(0, {a});")

"""

#this returns a list of elements that will be passed into another function
def job_listing_page():
    time.sleep(5)
    #vital component to code
    global driver
    
    global unique_list

    
    #master_element_list, list of lists
    master_element_list = []
    
    #introduce a for loop for scrolling and capturing lists of elements
    for pixel_scroll in range(0,5000,250):
        
        #sleep the program so that page can load properly
        time.sleep(1)

        #this section attempts to find a location on the page and then scroll down by the specified pixel amount
        try:
            
            # Find the element with class 'jobs-search-results-list', this will be the element that the program will locate
            # the program will go that area in the webpage and then scroll down by the pixel scroll amount
            #Find the element with class 'jobs-search-results-list'
            results = driver.find_element(By.CLASS_NAME,'jobs-search-results-list')

            # #string format for scrolling function where pixel_scroll is the amount to scroll by
            scroll_string = f"arguments[0].scrollBy(0, {pixel_scroll});"

            #scroll down, this may break if the results variable is not valid
            driver.execute_script(scroll_string, results)

            

        except:
            """
            This program is competing against the programs created LinkedIn software engineers
            the easy fix is pull the 
            """
            print('xpath for job listings changed')            
            
        
        
        #after it scrolls, then the program will attempt to capture all the web elements in its vicinity (before and after web position)
        try:
            
            #the job web elements are in the following xpath:
            #xpath for individual job listings
            job_xpath = "//ul[@class='scaffold-layout__list-container']//div[contains(@id,'ember')]//a[contains(@class,'job-card-list')]"
            
            #once in the job search page, get the job elements: this is the correct xpath
            job_elements = driver.find_elements(By.XPATH,job_xpath)

            #append to master list that will contain the job elements in a random order
            master_element_list.append(job_elements)

            
        #the program could not locate the web element that contains the list of several job elements 
        #that will be interacted with to apply for jobs
        except:
            print('job_elements list failed because of LinkedIn webpage element could not be located')
            
            
    #get the unique elements in the list of lists
    unique_list = []
    
    #mechanism to get the unique job elements in the order of the page (expect to see several duplicated) `
    for sublist in master_element_list:
        for element in sublist:
            if element not in unique_list:
                unique_list.append(element)
                
                
    #go to the beginning of the page
    #string format for scrolling function
    scroll_string = f"arguments[0].scrollBy(0, {-5000});"

    #Find the element with class 'jobs-search-results-list'
    results = driver.find_element(By.CLASS_NAME,'jobs-search-results-list')

    #scroll down
    driver.execute_script(scroll_string, results)

    #display how many jobs the program will attempt to apply to
    print('Number of jobs on page',len(unique_list))
    
    #this returns a list of job elements that will be the input for another function that attempts to apply
    return unique_list

In [4]:
#loop through job listings and apply where applicable
def loop_through_job_listings(job_list):
    
    
    #get a count of how many successful, unsuccesful, and total job applications:
    unsuccessful_application, successful_applications, total_applications = 0,0,0
        
    for job in job_list:
        
        #count the total amount of applications in the program
        total_applications = total_applications +1
        
        #click on the job title to take you the section where you can apply
        job.click()

        #sleep that bitch so that things can load properly
        time.sleep(3)

        #once inside the section find the easy apply button
        try:
            #this code takes you to the section where you can click the easy apply button
            easy_apply_xpath = "//div[@class='jobs-apply-button--top-card']"
            easy_apply_button = driver.find_element(By.XPATH,easy_apply_xpath)
            easy_apply_button.click()
            time.sleep(3)

        except:
            print('Previously submitted application to',job.text)
            continue 

        """
        after clicking the easy apply button, the application will begin with the user's contact information
        and resume (all pre-inputed)
        
        """
        try:
            #the first page will ask and contain contact information that is already specified by the user
            next_xpath = "//button[@aria-label='Continue to next step']"
            next_button = driver.find_element(By.XPATH,next_xpath)
            next_button.click()
            time.sleep(1)
        
        
        ###this except clause handles the applications that require you to just hit submit
        except:
 
            #chances are that if the first step failed, then its one of those applications that you just hit submit
            #attempt to submit the application
            try:
                unfollow_xpath = "//div[@class='job-details-easy-apply-footer__section']"
                unfollow_section = driver.find_element(By.XPATH, unfollow_xpath)
                unfollow_section.click()
                

            except:
                print('unfollow section does not work')

            #actually submit the application
            try:

                submit_xpath = "//button[@aria-label='Submit application']"
                submit_application_button = driver.find_element(By.XPATH, submit_xpath)
                submit_application_button.click()
                
                #this means the application was submitted, but doesn't mean things changed on website
                print(job.text)
                print('application submitted')
                successful_applications = successful_applications + 1

                time.sleep(2)
                
            except:
                print('submit application button broken')
            
            #theres a pop up that states you have submitted your application, exit out of it
            #it should be the first instance of that xpath
            try:
                submission_exit_path = "//button[@aria-label='Dismiss']"
                submission_exit_button = driver.find_element(By.XPATH,submission_exit_path)
                submission_exit_button.click()
                time.sleep(2)
                #continue to the next job in the list
                continue

            except:
                print('dismiss button didnt work after application was submitted') 
                continue
            
            


        try:
            #at this point we should be at the upload resume section
            #get the value of the progess percentage
            #if it is not equal to the 50 percent then skip to the next job
            progess_xpath = "//span[@class='pl3 t-14 t-black--light']"
            progress_element = driver.find_element(By.XPATH,progess_xpath)
            
#             print(progress_element.text)
#             print()
            time.sleep(2)

        except:
            print('progess not showing up')


        #we are only interested in the truly easy apply applications
        if progress_element.text == '50%':
            
            #click on the review resume button
            try:

                review_xpath = "//button[@aria-label='Review your application']"
                review_button = driver.find_element(By.XPATH, review_xpath)
                review_button.click()

            except:
                print('review button broke')

            #you don't want to follow these companies fuck that    
            try:
                unfollow_xpath = "//div[@class='job-details-easy-apply-footer__section']"
                unfollow_section = driver.find_element(By.XPATH, unfollow_xpath)
                unfollow_section.click()

            except:
                print('unfollow section does not work')

            #actually submit the application
            try:

                submit_xpath = "//button[@aria-label='Submit application']"
                submit_application_button = driver.find_element(By.XPATH, submit_xpath)
                submit_application_button.click()
                
                #this means the application was submitted, but doesn't mean things changed on website
                print('application submitted for',job.text)
                
                #add to successful count
                successful_applications = successful_applications + 1
                
                time.sleep(2)
                
            except:
                print('submit application button broken')

            #theres a pop up that states you have submitted your application, exit out of it
            #it should be the first instance of that xpath
            try:
                submission_exit_path = "//button[@aria-label='Dismiss']"
                submission_exit_button = driver.find_element(By.XPATH,submission_exit_path)
                submission_exit_button.click()
                
                #let the website load properly before moving on
                time.sleep(2)
                
                #continue to the next job in the list
                continue

            except:
                print('dismiss button didnt work after application was submitted')


        #if progress is not at 50 then close out the application
        else:

            close_out_xpath =  "//button[@aria-label='Dismiss']"
            close_out_button = driver.find_element(By.XPATH,close_out_xpath)
            close_out_button.click()
            time.sleep(2)


            try:
                discard_xpath = "//button[@data-control-name='discard_application_confirm_btn']"
                discard_button = driver.find_element(By.XPATH,discard_xpath)
                discard_button.click()
                
                #add to unsuccessful count
                unsuccessful_applications = unsuccessful_applications + 1
                
                time.sleep(5)
                continue
            except:
                continue    
                
    #create list that will return metrics     
    metric_list = [unsuccessful_application, successful_applications, total_applications]
       
    #return a list where the first position is the number of successful, unsuccessful, and total number of applications
    return metric_list

In [5]:
"""
function that will go to the other page
"""
def pagnation(page_number):
    
    #declare driver as global so that it connects with previous functions
    global driver
    
    #have the program scroll down to the bottom of the page
    pixel_scroll = 1250
    
    results = driver.find_element(By.CLASS_NAME,'jobs-search-results-list')

    # #string format for scrolling function where pixel_scroll is the amount to scroll by
    scroll_string = f"arguments[0].scrollBy(0, {pixel_scroll});"

    #scroll down, this may break if the results variable is not valid
    driver.execute_script(scroll_string, results)
            
            
    #this xpath changes depending on the page number of interest
    page_number_xpath = f"//button[@aria-label='Page {page_number}']"
    
    #element to interact with 
    page_number_button = driver.find_element(By.XPATH,page_number_xpath)
    
    #click the button
    page_number_button.click()
    
    #sleep to give program enough time to rerun
    time.sleep(1)

# Main Mechanism

### Populate the parameters to meet your ideal job application process

You will have to manually filter the job specifications on LinkedIn while you are signed into your personal account and then copy that URL link and store it into a variable. I have remote roles in data science, data analytics, and business analysis.

In [6]:
#specify the email/password
email_input = 'username'
password_input = 'password'

#different types of roles
ds_job_url = "https://www.linkedin.com/jobs/search/?currentJobId=3456954752&f_AL=true&f_WT=2&geoId=103644278&keywords=data%20science&location=United%20States&refresh=true"
analyst_job_url ='https://www.linkedin.com/jobs/search/?currentJobId=3521184843&f_AL=true&f_SB2=4&f_WT=2&keywords=analyst&refresh=true&sortBy=R&spellCorrectionEnabled=false'
business_analyst_job_url = 'https://www.linkedin.com/jobs/search/?currentJobId=3530135393&f_AL=true&f_JT=F&f_SB2=4&f_WT=2&geoId=103644278&keywords=business%20analyst&location=United%20States&refresh=true'

#sign in to linkedin and get to the job listing of interest (page 1)
linkedInSignIn(email_input,password_input,analyst_job_url)


In [8]:
"""
mechanism that logs into linkedIn, goes to the job listing page of interest, applies to jobs that are easy apply,
and keeps on clicking on pages until the for loop ends.

"""
print('Page 1')

#counter of metrics
bad,good,total = 0,0,0

#begins at 2 because the program automatically goes to the first page
for job_listing_page_number in range(2,5):
    


    #go to the job listing page of interest and get the elements pertaining to the jobs on that page
    job_listings_on_linkedIn = job_listing_page()

    #run function to have the program apply for jobs automatically, this will return the number of successful applications
    #will also contain the number of unsuccessful applications and the total number of applications across all pages being loopped
    performance_metrics = loop_through_job_listings(job_listings_on_linkedIn)
    
    #assing total values after looping through all the 
    bad = bad + performance_metrics[0]
    good = good + performance_metrics[1]
    total = total + performance_metrics[-1]
    
    print('Unsuccessful: ',bad,'\n','Successful: ',good,'\n','Total: ',total)
    print('Success Rate: ', round((good/total)*100,4))
    print()
    
    #once the program is done applying for applicable easy apply jobs, then 
    pagnation(job_listing_page_number)
    
    print('Page:  ', job_listing_page_number)



Page 1
Number of jobs on page 21
application submitted for Quantitative Analyst Quant - Global Investment Bank
Unsuccessful:  0 
 Successful:  1 
 Total:  21
Success Rate:  4.7619

Page:   2
Number of jobs on page 7
application submitted for Senior Financial Business Analyst
application submitted for Integration Developer Analyst, Sr
Unsuccessful:  0 
 Successful:  3 
 Total:  28
Success Rate:  10.7143

Page:   3
Number of jobs on page 7
Sr. GIS Analyst - 113
application submitted
application submitted for Senior Actuarial Reserving Analyst
Unsuccessful:  0 
 Successful:  5 
 Total:  35
Success Rate:  14.2857

Page:   4
